In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy import stats
from sqlalchemy import create_engine
import warnings
from scipy.stats.mstats import winsorize
from sklearn.model_selection import train_test_split

In [42]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

student_df = pd.read_sql_query('select * from studentsperformance',con=engine)

# No need for an open connection, 
# because you're only doing a single query
engine.dispose()
student_df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [43]:
student_df.isna().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [54]:
X=student_df.drop(columns=['math score','reading score','writing score'])
X=pd.get_dummies(X,drop_first=True)
Y=student_df['math score']

In [55]:

X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size=0.2)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((800, 12), (200, 12), (800,), (200,))

In [46]:
# KNN regression
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor(n_neighbors=20)
knn.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                    weights='uniform')

In [47]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(knn,X_train,y_train,cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))


Accuracy: 0.16 (+/- 0.08)


In [48]:
y_pred=knn.predict(X_test)

In [49]:
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse,rmse
print('mean_absolute_error: {}'.format(mean_absolute_error(y_test,y_pred)))
print('mse: {}'.format(mse(y_test,y_pred)))
print('rmse: {}'.format(rmse(y_test,y_pred)))

mean_absolute_error: 10.774499999999998
mse: 191.12517499999998
rmse: 13.8248028919041


In [58]:
# OLS regression
import statsmodels.api as sm
X_train=sm.add_constant(X_train)
result=sm.OLS(y_train,X_train).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             math score   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     24.29
Date:                Mon, 06 Sep 2021   Prob (F-statistic):           1.77e-46
Time:                        20:00:50   Log-Likelihood:                -3186.3
No. Observations:                 800   AIC:                             6399.
Df Residuals:                     787   BIC:                             6459.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            58.0396      2.223     26.111      0.000      53.676      62.403
gender_male                                       4.8363      0.937      5.163      0.000       2.998       6.675
race/ethnicity_group B                            2.3601      1.989      1.187      0.236      -1.543       6.263
race/ethnicity_group C                            2.9399      1.888      1.557      0.120      -0.767       6.646
race/ethnicity_group D                            6.1309      1.909      3.211      0.001       2.383       9.879
race/ethnicity_group E                            9.8288      2.106      4.667      0.000       5.695      13.962
parental level of education_bachelor's degree     2.5159      1.670      1.506      0.132      -0.762       5.794
parental level of education_high school          -6.0144      1.462     -4.115      0.000      -8.884      -3.145
parental level of education_master's degree       2.6663      2.152      1.239      0.216      -1.558       6.890
parental level of education_some college         -0.8793      1.385     -0.635      0.526      -3.598       1.839
parental level of education_some high school     -4.0970      1.516     -2.702      0.007      -7.073      -1.121
lunch_standard                                   10.6706      0.966     11.043      0.000       8.774      12.567
test preparation course_none                     -5.8903      0.976     -6.038      0.000      -7.805      -3.975
==============================================================================
Omnibus:                        8.057   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                7.132
Skew:                          -0.170   Prob(JB):                       0.0283
Kurtosis:                       2.687   Cond. No.                         13.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
X_test=sm.add_constant(X_test)
y_pred=result.predict(X_test)

/Users/pegahafshar/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/pegahafshar/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/pegahafshar/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/pegahafshar/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/pegahafshar/opt/anaconda3/lib

In [61]:
print('mean_absolute_error: {}'.format(mean_absolute_error(y_test,y_pred)))
print('mse: {}'.format(mse(y_test,y_pred)))
print('rmse: {}'.format(rmse(y_test,y_pred)))

mean_absolute_error: 10.707681075377458
mse: 183.71723033423882
rmse: 13.554232930499564


In KNN model R_squered is 0.16 and in OLS R_squered is 0.255 which is higher. However both models are poor but OLS is better than KNN in training.
In test set OLS has more errors than KNN.